In [1]:
from dependency import *
from resnet18.resnet_model import Model
import tensorflow as tf
from tf_utils import tf_train, tf_test_error_rate, set_model_flags
import matplotlib.pyplot as plt
from data_utils import dataset
import os
from load_data import load_images
import tiny_imagenet_loader

Using TensorFlow backend.


In [2]:
def adv_model_resnet():
    graph = tf.Graph()
    with graph.as_default():
        images = tf.placeholder(tf.float32, (None, 64, 64, 3))

        # preprocessing
        _R_MEAN = 123.68
        _G_MEAN = 116.78
        _B_MEAN = 103.94
        _CHANNEL_MEANS = [_R_MEAN, _G_MEAN, _B_MEAN]
        features = images - tf.constant(_CHANNEL_MEANS)

        model = Model(
            resnet_size=18,
            bottleneck=False,
            num_classes=200,
            num_filters=64,
            kernel_size=3,
            conv_stride=1,
            first_pool_size=0,
            first_pool_stride=2,
            second_pool_size=7,
            second_pool_stride=1,
            block_sizes=[2, 2, 2, 2],
            block_strides=[1, 2, 2, 2],
            final_size=512,
            version=2,
            data_format=None)

        logits = model(features, False)
        
        preds = tf.nn.softmax(logits)

        saver = tf.train.Saver()

    return graph, saver, images, logits, preds


In [3]:
def error_rate(predictions, labels):
    """
    Return the error rate in percent.
    """

    assert len(predictions) == len(labels)
    #print("Predictions:", np.argmax(predictions, 1))
    #print("Labels:", np.argmax(labels, 1))

    # return 100.0 - (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])
    return 100.0 - (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / len(predictions))





In [4]:
#set_model_flags(False)

graph, saver, images, logits, preds = adv_model_resnet()

sess = tf.Session(graph = graph)
#sess.run(tf.global_variables_initializer())
#model.tf_load(sess, "./resnet18/checkpoints/model/")

path = os.path.join('resnet18', 'checkpoints', 'model')
saver.restore(sess, tf.train.latest_checkpoint(path))

with sess.as_default():
    
    #data = dataset('../Defense_Model/tiny-imagenet-200/train', '../Defense_Model/tiny-imagenet-200/val', normalize = False)
    #batch_data, batch_labels = data.next_valid_batch(256)
    #X_train, Y_train, 
    data = tiny_imagenet_loader.TinyImageNetLoader('val', "../Defense_Model/tiny-imagenet-200/")
    X_test, Y_test = data.load_n_images_all_classes(2)
    feed_dict = {
                            images: X_test
                            #train_mode: True
                }
    fetches = preds
    test_preds = sess.run(fetches=fetches, feed_dict=feed_dict)
    #print(sess.run(tf.get_variable("conv2d_2/kernel")))
    err = error_rate(test_preds, Y_test)

INFO:tensorflow:Restoring parameters from resnet18/checkpoints/model/model.ckpt-5865


In [5]:
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file


# List ALL tensors example output: v0/Adam (DT_FLOAT) [3,3,1,80]
print_tensors_in_checkpoint_file(file_name=tf.train.latest_checkpoint(path), tensor_name='', all_tensors=True)

tensor_name:  batch_normalization/beta
[ 0.15327251 -3.6648762   0.29602697 -5.5095787   0.556029    0.5861532
  0.39628828  0.37059164  1.0624574  -0.7373112  -3.31729     0.4853037
 -0.28816277  0.41837376  1.5063534   0.25703     0.84342736 -1.661739
 -3.5054302  -2.0207784   0.24919432  1.3076822   0.20476404 -0.8423358
  0.23986575  0.8820192   0.24003492  0.38961437  0.27521265 -0.67344093
 -0.05812984 -1.8170879  -4.560499    0.2559829   2.0706015  -5.0386915
  0.7056391   3.4599018  -3.3357253   0.30525696  0.39492506  1.1428201
  0.69456947 -1.8349607   0.3913201  -2.2377534  -2.632812    0.9325869
 -0.03759874  0.4301127   0.74933565 -2.9703825  -0.14223686  1.2867179
  1.1761544   0.9584662   0.5240119   0.6302699   0.5364666   3.0697289
  1.3128197   1.0684016  -5.820258   -2.238164  ]
tensor_name:  batch_normalization/beta/Momentum
[-5.2513960e-03 -5.8927694e-03 -2.7822560e-02  6.5759262e-03
 -2.7928604e-03 -2.9188516e-02  1.6352084e-03 -8.2016811e-03
  4.7918243e-04 -7.63

  3.35918739e-04 -4.54348919e-04  2.86607625e-04 -2.74706073e-03]
tensor_name:  batch_normalization_16/gamma
[5.823392  4.9682293 4.913923  5.311611  5.541503  5.5467453 5.0600395
 4.725688  5.3352604 5.9073753 5.6062098 4.5677433 6.0048337 5.714098
 5.3891673 5.088279  4.4569488 5.832358  5.5421453 5.0647836 5.469647
 5.2833285 5.118003  5.828763  6.0187287 4.837193  5.8452463 4.895041
 5.422034  5.7647185 5.0190716 4.8597784 5.2677054 5.076386  5.497064
 5.231054  5.9743986 5.678292  5.252689  5.394384  5.2674985 5.2706633
 5.5096607 4.9730783 5.201558  5.2941566 4.8287888 5.3650074 5.094759
 4.9130206 5.4183936 5.17703   4.7401586 4.912799  5.744903  4.4912205
 4.9974384 5.3127823 5.539597  5.644943  5.4856405 4.991393  5.402684
 5.1615715 5.7714744 5.350355  5.0070696 4.0902696 5.2838883 5.1554494
 6.339528  5.148129  5.4026766 5.4026113 5.400179  6.0609035 5.622102
 5.9763484 4.9349566 5.690851  4.9600425 5.0629153 5.0665107 5.05729
 5.683146  5.8409023 5.697801  5.003789  5.22727

[[[[-0.00385295  0.00088858  0.00341512 ...  0.01842232 -0.0159253
    -0.00624101]
   [ 0.01079691  0.0044373   0.04128841 ... -0.01878277  0.0089697
     0.02006797]
   [-0.0164857  -0.01292062 -0.02443952 ... -0.00075618  0.00248148
    -0.00730072]
   ...
   [-0.01154529 -0.00713475 -0.00919911 ... -0.00402696 -0.00771272
    -0.01286934]
   [-0.05449753  0.00964724  0.01000245 ... -0.00857048 -0.01631087
    -0.02289186]
   [ 0.00513295 -0.02795807 -0.00367223 ... -0.0027904   0.0050041
     0.0345163 ]]

  [[-0.01403826  0.00718312  0.01069201 ...  0.02050789 -0.01912175
    -0.00364036]
   [ 0.01286561 -0.00561456  0.00704762 ... -0.02158314 -0.00239343
     0.02591834]
   [ 0.00574733 -0.0136973  -0.01987895 ... -0.01084665  0.01629505
    -0.00491008]
   ...
   [ 0.04329995  0.00088957  0.01119404 ...  0.00143086 -0.03205395
     0.01938709]
   [-0.04468801  0.002496   -0.01195268 ... -0.00059435 -0.01200487
    -0.00587686]
   [ 0.01291895 -0.02415512  0.0038928  ...  0.01790

     1.02824229e-03 -1.22952778e-02]]]]
tensor_name:  conv2d_20/kernel
[[[[-1.15664089e-02 -4.90469020e-03 -5.89930546e-03 ... -2.88517382e-02
    -1.27282143e-02 -9.66925453e-03]
   [ 1.18726017e-02 -9.03578009e-03 -1.14594800e-02 ... -3.54104722e-03
    -6.32703491e-03  5.49907237e-03]
   [-2.55987025e-03 -1.45836389e-02 -7.05167558e-03 ...  1.03524104e-02
     1.41128795e-02 -1.16443001e-02]
   ...
   [ 1.52619006e-02  7.59039633e-03  3.91538779e-04 ... -8.10709083e-04
     9.05998424e-03  7.51177920e-03]
   [ 9.00583249e-03 -5.92825003e-03  7.59074371e-03 ... -1.26173859e-02
     4.19613952e-03 -7.11005367e-03]
   [-9.32666240e-04 -4.56281286e-03 -6.97525172e-03 ... -2.54884642e-02
    -8.55305698e-03  1.47891967e-02]]

  [[-8.80793110e-03 -1.31464563e-02  9.09569487e-03 ... -2.80256830e-02
    -1.22117046e-02 -1.13508338e-02]
   [ 6.08386286e-03 -1.86452351e-03 -1.36276437e-02 ... -2.91659054e-03
    -1.54009857e-03 -1.46605528e-03]
   [ 6.37631048e-04 -1.30387945e-02 -1.11119235e

In [6]:
err

21.75

In [7]:
len(var)

NameError: name 'var' is not defined

In [ ]:
var

In [ ]:
np.argmax(batch_labels, 1)[:10]

In [ ]:
np.argmax(test_preds, 1)[:10]

In [ ]:
np.sum(np.argmax(batch_labels, 1) == np.argmax(test_preds, 1))

In [ ]:

import matplotlib.pyplot as plt
f, a = plt.subplots(1, 1, figsize=(64, 64))
a.imshow(batch_data[0])

plt.show()
batch_data[0]

In [ ]:
from scipy.misc import imread
filename = "/Users/wendy/Documents/2018Summer/deepLearningAndprivacy/AdversialAttack/Defense_Model/tiny-imagenet-200/test/images/test_755.JPEG"
#file = tf.read_file(filename)
#image = tf.image.decode_jpeg(file, channels=3)
image = imread("/Users/wendy/Documents/2018Summer/deepLearningAndprivacy/AdversialAttack/Defense_Model/tiny-imagenet-200/test/images/test_755.JPEG", 
              mode='RGB')

In [ ]:
image

In [ ]:

imgplot = plt.imshow(image)